In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.ensemble import RandomForestClassifier
import time
import xgboost as xgb
import lightgbm as lgb
import warnings
import pickle

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
plt.style.use(style='seaborn')
%matplotlib inline

In [2]:
%%time
target = pd.read_csv('target.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

Wall time: 33.3 s


In [3]:
test_id = pd.read_csv('test2.csv')
test_id = test_id['TransactionID']

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [5]:
train = StandardScaler().fit_transform(train)

In [6]:
%%time
# XGBOOST
xgmodel = xgb.XGBClassifier(
    n_estimators=300,
    bagging_fraction = 0.8993155305338455,
    colsample_bytree = 0.7463058454739352,
    feature_fraction = 0.7989765808988153,
    gamma = 0.6665437467229817,
    learning_rate = 0.013887824598276186,
    max_depth = 16,
    min_child_samples = 170,
    num_leaves = 220,
    reg_alpha = 0.39871702770778467,
    reg_lambda = 0.24309304355829786,
    subsample = 0.7
)

xgmodel.fit(train, target)

[02:23:17] WARNING: ..\src\learner.cc:541: 
Parameters: { bagging_fraction, feature_fraction, min_child_samples, num_leaves } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[02:23:35] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 1h 12min 26s


XGBClassifier(bagging_fraction=0.8993155305338455, base_score=0.5,
              booster='gbtree', colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.7463058454739352,
              feature_fraction=0.7989765808988153, gamma=0.6665437467229817,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.013887824598276186, max_delta_step=0,
              max_depth=16, min_child_samples=170, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=300,
              n_jobs=4, num_leaves=220, num_parallel_tree=1, random_state=0,
              reg_alpha=0.39871702770778467, reg_lambda=0.24309304355829786,
              scale_pos_weight=1, subsample=0.7, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [9]:
y_pred_xg = xgmodel.predict_proba(test)

In [8]:
sub = pd.DataFrame()
sub['TransactionID'] = test_id
sub['isFraud'] = y_pred_xg[:, 1]
sub.to_csv('3submission_xgb.csv', index=False)